In [223]:
import sys
sys.path.append("../")  # 添加项目根目录到路径中

In [224]:
import torch
import torch.optim as optim
from torch import nn

import hgp
from hgp.models import HGNNP,CHGNN
from hgp.function import StraightThroughEstimator

import numpy as np
import random
DEVICE = torch.device("cuda:1") if torch.cuda.is_available() else torch.device("cpu")
DEVICE


device(type='cuda', index=1)

In [225]:
seed = 42
torch.manual_seed(seed) # 为CPU设置随机种子
torch.cuda.manual_seed(seed) # 为当前GPU设置随机种子
torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU，为所有GPU设置随机种子
np.random.seed(seed)  # Numpy module.
random.seed(seed)  # Python random module.	
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [226]:
from hgp.models import ParameterDict

# fmt: off
h_hyper_prmts = ParameterDict()
l_hyper_prmts = ParameterDict()

partitions = 3
#1304
weight = 1

h_hyper_prmts["convlayers11"] = {"in_channels": 327, "out_channels": 327, "use_bn": True, "drop_rate": 0.2}
h_hyper_prmts["convlayers14"] = {"in_channels": 327, "out_channels": 327, "use_bn": True, "drop_rate": 0.05}
# h_hyper_prmts["convlayers12"] = {"in_channels": 327, "out_channels": 327, "use_bn": True, "drop_rate": 0.05}
# h_hyper_prmts["convlayers13"] = {"in_channels": 327, "out_channels": 327, "use_bn": True, "drop_rate": 0.05}
# h_hyper_prmts["convlayers15"] = {"in_channels": 327, "out_channels": 327, "use_bn": True, "drop_rate": 0.05}
# h_hyper_prmts["convlayers16"] = {"in_channels": 327, "out_channels": 327, "use_bn": True, "drop_rate": 0.05}
# h_hyper_prmts["convlayers19"] = {"in_channels": 327, "out_channels": 327, "use_bn": True, "drop_rate": 0.05}
# h_hyper_prmts["convlayers18"] = {"in_channels": 327, "out_channels": 327, "use_bn": True, "drop_rate": 0.05}
# h_hyper_prmts["convlayers10"] = {"in_channels": 327, "out_channels": 327, "use_bn": True, "drop_rate": 0.05}
# h_hyper_prmts["convlayers21"] = {"in_channels": 327, "out_channels": 327, "use_bn": True, "drop_rate": 0.05}
# h_hyper_prmts["convlayers22"] = {"in_channels": 327, "out_channels": 327, "use_bn": True, "drop_rate": 0.05}
# h_hyper_prmts["convlayers23"] = {"in_channels": 327, "out_channels": 327, "use_bn": True, "drop_rate": 0.05}
# h_hyper_prmts["convlayers24"] = {"in_channels": 327, "out_channels": 327, "use_bn": True, "drop_rate": 0.05}
# h_hyper_prmts["convlayers25"] = {"in_channels": 327, "out_channels": 327, "use_bn": True, "drop_rate": 0.05}
h_hyper_prmts["convlayers26"] = {"in_channels": 327, "out_channels": 327, "use_bn": True, "drop_rate": 0.05}
h_hyper_prmts["convlayers17"] = {"in_channels": 327, "out_channels": 1024, "use_bn": True, "drop_rate": 0.05}

# l_hyper_prmts["linerlayer13"] = {"in_channels":1024, "out_channels":1024, "use_bn":True, "drop_rate":0.05}
l_hyper_prmts["linerlayer1"] = {"in_channels":1024, "out_channels":512, "use_bn":True, "drop_rate":0.05}
l_hyper_prmts["linerlayer12334"] = {"in_channels":512, "out_channels":256, "use_bn":True, "drop_rate":0.05}
# l_hyper_prmts["linerlayer12"] = {"in_channels":256, "out_channels":256, "use_bn":True, "drop_rate":0.05}
# l_hyper_prmts["linerlayer123"] = {"in_channels":256, "out_channels":256, "use_bn":True, "drop_rate":0.05}
l_hyper_prmts["linerlayer121"] = {"in_channels":256, "out_channels":64, "use_bn":True, "drop_rate":0.05}
l_hyper_prmts["linerlayer1w2"] = {"in_channels":64, "out_channels":32, "use_bn":True, "drop_rate":0.05}
l_hyper_prmts["linerlayer31"] = {"in_channels":32, "out_channels":3, "use_bn":False, "drop_rate":0.05}



hyper = {
    "h_hyper_prmts": h_hyper_prmts,
    "l_hyper_prmts":l_hyper_prmts,
    "init_features_dim":list(h_hyper_prmts.values())[0]["in_channels"],
    "partitions":partitions
}

# fmt: on

In [227]:
def loss_bs_matrix(outs, hg, device,weight):
    # fmt: off
    r"""
    对于超图的损失函数的矩阵形式.
    
    Args:
        ``outs``(`torch.nn.Module`):  模型的输出. Size :math:`(N, nums_classes)`.   
        ``hg``(`Hypergraph`):  超图对象.  
    """
    # fmt: on
    H = hg.H.to_dense().to(device)
    outs = outs.to(device)
    nn = torch.matmul(outs, (1 - torch.transpose(outs, 0, 1)))
    ne_k = torch.matmul(nn, H)
    ne_k = ne_k.mul(H)

    H_degree = torch.sum(H, dim=0)
    H_degree = H_degree

    H_1 = ne_k / H_degree
    a2 = 1 - H_1
    a3 = torch.prod(a2, dim=0)
    a3 = a3.sum()
    loss_1 = -1 * a3

    # pun = torch.mul(ne_k, H)

    # loss_1 = pun.sum()
    loss_2 = torch.var(torch.sum(outs, dim=0)).to(device)
    loss =  weight * loss_1 + loss_2
    return loss, loss_1, loss_2

#### 定义用于训练的类Trainer

In [228]:
class Trainer(nn.Module):
    # fmt: off
    r"""
    用于承担训练的类.
    ---
    Args:
        ``net``: (``torch.nn.Module``): 网络模型.  
        ``X``: (``torch.Tensor``): 作为输入的顶点特征矩阵. Size :math:`(N, C_{in})`.  
        ``hg``: (``dhg.Hypergraph``): 包含 :math:`N` 个顶点的超图结构.  
    """
    # fmt: on
    def __init__(self, net, X, hg, optimizer):
        super().__init__()
        self.X: torch.Tensor = X.to(DEVICE)
        self.hg = hg.to(DEVICE)
        self.de = self.hg.H.to_dense().sum(dim=0).to("cpu").to(DEVICE)
        self.optimizer1: torch.optim.Optimizer = optimizer
        self.optimizer2: torch.optim.Optimizer = optimizer
        self.convlayers = nn.ModuleList()
        self.convlayers.append(net.to(DEVICE))
        self.linearlayers = nn.ModuleList()
        self.weight = 200
        
    def forward(self, X):
        for layer in self.convlayers:
            X = layer(X, self.hg)
        for layer in self.linearlayers:
            if isinstance(layer, nn.MultiheadAttention):
                X,_ = layer(X, X, X)
            else:
                X = layer(X)
        return X

    def run(self, epoch):
        self.train()
        self.optimizer1.zero_grad()
        self.optimizer2.zero_grad()
        outs = self.forward(self.X)
        loss, loss_1, loss_2 = loss_bs_matrix(outs, self.hg, device=DEVICE,weight=self.weight)
        loss.backward()
        self.optimizer1.step()
        self.optimizer2.step()
        
        return loss.item(), loss_1.item(), loss_2.item()

#### 准备数据

In [229]:
import hgp.utils
G = hgp.utils.from_pickle_to_hypergraph("../data/high")
edges, _ = G.e
G.num_e,G.num_v

(7818, 327)

In [230]:
X = torch.randn(size=(G.num_v, hyper["init_features_dim"]))
# X = torch.eye(hyper["init_features_dim"])
net = HGNNP(hyper["h_hyper_prmts"]).to(DEVICE)
hgnn_trainer = Trainer(net=net, X=X, hg=G, optimizer=None).to(DEVICE)
for (k,v) in hyper["l_hyper_prmts"].items():
    if str.startswith(k,"attn"):
        hgnn_trainer.linearlayers.append(nn.MultiheadAttention(embed_dim=v["in_channels"], num_heads=2).to(DEVICE))
    else:
        hgnn_trainer.linearlayers.append(nn.BatchNorm1d(num_features=v["in_channels"]).to(DEVICE)) if v["use_bn"] else None
        hgnn_trainer.linearlayers.append(nn.ReLU().to(DEVICE))
        if v["drop_rate"] > 0:
            hgnn_trainer.linearlayers.append(nn.Dropout(v["drop_rate"]))
        hgnn_trainer.linearlayers.append(nn.Linear(in_features=v["in_channels"],out_features=v["out_channels"],device=DEVICE))
hgnn_trainer.linearlayers.append(nn.Softmax(dim=1))

ModuleList(
  (0): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (1): ReLU()
  (2): Dropout(p=0.05, inplace=False)
  (3): Linear(in_features=1024, out_features=512, bias=True)
  (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU()
  (6): Dropout(p=0.05, inplace=False)
  (7): Linear(in_features=512, out_features=256, bias=True)
  (8): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (9): ReLU()
  (10): Dropout(p=0.05, inplace=False)
  (11): Linear(in_features=256, out_features=64, bias=True)
  (12): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (13): ReLU()
  (14): Dropout(p=0.05, inplace=False)
  (15): Linear(in_features=64, out_features=32, bias=True)
  (16): ReLU()
  (17): Dropout(p=0.05, inplace=False)
  (18): Linear(in_features=32, out_features=3, bias=True)
  (19): Softmax(dim=1)
)

In [231]:
# hgnn_trainer.layers
# for n,p in hgnn_trainer.named_parameters():
#     print(n,p)
hgnn_trainer.weight = weight

In [232]:
temp_loss_total,temp_loss1,temp_loss2 = torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False)
optim1 = optim.Adam(hgnn_trainer.convlayers.parameters(), lr=4e-4, weight_decay=5e-8)
optim2 = optim.Adam(hgnn_trainer.linearlayers.parameters(), lr=4e-3, weight_decay=5e-8)
hgnn_trainer.optimizer1 = optim1
hgnn_trainer.optimizer2 = optim2
for epoch in range(20000):
    if hgnn_trainer.weight > 0.0366:
        hgnn_trainer.weight = hgnn_trainer.weight - 0.00045
    loss,loss_1,loss_2 = hgnn_trainer.run(epoch=epoch)
    temp_loss_total += loss
    temp_loss1 += loss_1
    temp_loss2 += loss_2
    if epoch % 10 == 0:
        print(f"in {epoch} epoch, average loss: {temp_loss_total.item() / 10}")
        print(f"                , loss1: {temp_loss1.item() / 10}")
        print(f"                , loss2: {temp_loss2.item() / 10}")
        print(f"                , loss2_weight: {hgnn_trainer.weight}")
        print(f"=================================")
        sys.stdout.flush()
        temp_loss_total,temp_loss1,temp_loss2 = torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False)

in 0 epoch, average loss: -69.18345336914062
                , loss1: -70.0277099609375
                , loss2: 0.8127448081970214
                , loss2_weight: 0.99955


in 10 epoch, average loss: -1528.928125
                , loss1: -1603.15791015625
                , loss2: 68.797509765625
                , loss2_weight: 0.9950500000000005
in 20 epoch, average loss: -4004.894921875
                , loss1: -4061.073046875
                , loss2: 24.85688934326172
                , loss2_weight: 0.990550000000001
in 30 epoch, average loss: -6510.801171875
                , loss1: -6647.296875
                , loss2: 56.6980712890625
                , loss2_weight: 0.9860500000000015
in 40 epoch, average loss: -7014.4734375
                , loss1: -7210.98984375
                , loss2: 78.0598876953125
                , loss2_weight: 0.981550000000002
in 50 epoch, average loss: -7011.1734375
                , loss1: -7228.859375
                , loss2: 66.41965942382812
                , loss2_weight: 0.9770500000000025
in 60 epoch, average loss: -6991.66328125
                , loss1: -7232.55234375
                , loss2: 57.00101318359375
   

KeyboardInterrupt: 

In [ ]:
hgnn_trainer.eval()
outs = hgnn_trainer.forward(hgnn_trainer.X)
outs_straight = StraightThroughEstimator.apply(outs)
G_clone = G.clone()
edges, _  = G_clone.e
cut = 0
for vertices in edges:
    if torch.prod(outs_straight[list(vertices)], dim=0).sum() == 0:
        cut += 1
    else:
        G_clone.remove_hyperedges(vertices)
assert cut == G_clone.num_e
cut

954

In [ ]:
num_nodes = outs_straight.sum(dim=0)
print(num_nodes)
(torch.max(num_nodes).item() - torch.min(num_nodes).item()) / num_nodes.sum().item()


tensor([108., 110., 109.], device='cuda:1', grad_fn=<SumBackward1>)


0.0061162079510703364